### Mean Square Displacements From an Elastic Scan

Fist, here's the full script:

In [ ]:
from __future__ import (absolute_import, division, print_function)

from os.path import join
import numpy as np
from mantid.simpleapi import (BASISReduction, GroupWorkspaces, ElasticWindowMultiple,
                              LoadNexus, MSDFit, SaveNexus)
from mantid.kernel import ConfigService

# we set the default save directory to "/tmp"
config = ConfigService()
config['defaultsave.directory'] = '/tmp'

###################################################################################
## we reduce one set of runs from 83505 to 83536 corresponding to one elastic scan
###################################################################################
redution_options = dict(RunNumbers='83505-83536',
                                      DoIndividual=True,
                                      ReflectionType='silicon111',
                                      EnergyBins=[-100, 0.4, 100],
                                      MomentumTransferBins=[0.3, 0.2, 1.9],
                                      MaskFile='/SNS/BSS/shared/autoreduce/new_masks_08_12_2015/BASIS_Mask_default_111.xml')
BASISReduction(**redution_options)  # http://docs.mantidproject.org/nightly/algorithms/BASISReduction-v1.html

# Alternatively, we can load the reduced files if we had then under some directory:
"""
data_dir = '/tmp'  # assume the reduced files are in /tmp directory
names =  ['BSS_{number}_sqw'.format(number=r) for r in range(83505, 83536+1)]  # BSS_83505_sqw, ...
nexus_file_names = [name +  '.nxs' for name in names]  # BSS_83505_sqw.nxs, ...
nexus_files = [join(data_dir, name) for name in nexus_file_names]
for file, name in zip(nexus_files, names):
    LoadNexus(file, OutputWorkspace=name)
"""

###################################################
## Intensity decay vesus Temperature and versus Q
###################################################
spectra = ['BSS_{number}_sqw'.format(number=r) for r in range(83505, 83536+1)]  # BSS_83505_sqw, ...
GroupWorkspaces(InputWorkspaces=spectra, OutputWorkspace='grouped_spectra')
elwin_options=dict(InputWorkspaces='grouped_spectra',
                               IntegrationRangeStart=-0.0034,  # -3.4 micro-eV
                               IntegrationRangeEnd=0.0034,
                               BackgroundRangeStart=-0.1,
                               BackgroundRangeEnd=-0.03,
                               SampleEnvironmentLogName='SensorA',  # logs the temperature
                               SampleEnvironmentLogValue='average',
                               OutputInQ='intensity_vs_Q',  # intensity decay versus Q, at different temperatures
                               OutputInQSquared='intensity_vs_Q2',
                               OutputELF='intensity_vs_T',  # decay versus T, at different Q's
                               OutputELT='intensity_vs_T_normalized'    # decay versus T, at different Q's. Normalized to run with lowest T
                               )
ElasticWindowMultiple(**elwin_options)  # http://docs.mantidproject.org/nightly/algorithms/ElasticWindowMultiple-v1.html

#############
## MSD fit
###########
msdfit_options = dict(InputWorkspace='intensity_vs_Q',
                                   Model='Gauss',
                                   XStart=0.3,
                                   XEnd=1.9,
                                   SpecMin=0,
                                   SpecMax=83536 - 83505,
                                   OutputWorkspace='msdfit_summary',
                                   ParameterWorkspace='msdfit_params',  # contains MSD values, among other data
                                   FitWorkspaces='msdfit_curves'
                                   )
MSDFit(**msdfit_options)  # http://docs.mantidproject.org/nightly/algorithms/MSDFit-v1.html

##################################
## Save Workspaces to Nexus Files
##################################
SaveNexus(InputWorkspace='msdfit_summary', Filename='/tmp/msdfit_summary.nxs')
SaveNexus(InputWorkspace='msdfit_params', Filename='/tmp/msdfit_params.nxs')
SaveNexus(InputWorkspace='msdfit_curves', Filename='/tmp/msdfit_curves.nxs')

You can either:
- run the previous script if you are working in the [jupyter sns server](https://jupyter.sns.gov/hub/login)
- copy the script, then open MantidPlot and paste the script in the `Script Window`

Below we comment on some parts of the script:

In [ ]:
# we set the default save directory to "/tmp"
config = ConfigService()
config['defaultsave.directory'] = '/tmp'

Here we instruct Mantid to save any produced file under directory `/tmp`, unless we provide an absolute saving path
<pre>
SaveNexus(some_workspace, 'some_file.nxs')  # will save as /tmp/some_file.nxs
SaveNexus(some_workspace, '/home/abc/some_file.nxs')  # will save as /home/abc/some_file.nxs
</pre>

In [ ]:
###################################################
## Intensity decay vesus Temperature and versus Q
###################################################
spectra = ['BSS_{number}_sqw'.format(number=r) for r in range(83505, 83536+1)]  # BSS_83505_sqw, ...
GroupWorkspaces(InputWorkspaces=spectra, OutputWorkspace='grouped_spectra')
elwin_options=dict(InputWorkspaces='grouped_spectra',
                               IntegrationRangeStart=-0.0034,  # -3.4 micro-eV
                               IntegrationRangeEnd=0.0034,
                               BackgroundRangeStart=-0.1,
                               BackgroundRangeEnd=-0.03,
                               SampleEnvironmentLogName='SensorA',  # logs the temperature
                               SampleEnvironmentLogValue='average',
                               OutputInQ='intensity_vs_Q',  # intensity decay versus Q, at different temperatures
                               OutputInQSquared='intensity_vs_Q2',
                               OutputELF='intensity_vs_T',  # decay versus T, at different Q's
                               OutputELT='intensity_vs_T_normalized'    # decay versus T, at different Q's. Normalized to run with lowest T
                               )
ElasticWindowMultiple(**elwin_options)  # http://docs.mantidproject.org/nightly/algorithms/ElasticWindowMultiple-v1.html

This above code calculates the decay of the elastic intensity versus `Q` for a particular temperature, and also versus temperature for a particular `Q`.

- Algorithm [ElasticWindowMultiple](http://docs.mantidproject.org/nightly/algorithms/ElasticWindowMultiple-v1.html) requires to group all workspaces resulting from the previous reduction step under a `GroupWorkspace`, hence the need to call algorithm
[GroupWorkspaces](http://docs.mantidproject.org/nightly/algorithms/GroupWorkspaces-v1.html) first.
- For each spectra, we integrate the intensity in the range of energies corresponding to the width of the elastic peak. For Basis, it is <code>[-3.4, 3.4]</code> micro-eV.
- Each reduced run retains the value of the run temperature in the logs of the corresponing workspace, under log entry with name `SensorA`. 
- The temperature fluctuates during the time it takes to produce one run. We will use the average as representative value of the temperature for that particular run.

Four different workspaces are produced by `ElasticWindowMultiple`:
- `intensity_vs_Q`: Decay of elastic intensity versus Q. This workspace contains one spectrum for each run, or in other words, one spectrum per temperature.
- `intensity_vs_Q2`: same as `intensity_vs_Q` except that the decay is versus $Q^2$
- `intensity_vs_T`: Decay of elastic intensity versus temperature. This workspace contains one spectrum for each value of $Q$.
- `intensity_vs_T_normalized` Same as `intensity_vs_T`, but the value of the intensity is set to one for the lowest temperature. It's just a rescaling such that the intensity becomes exactly one for the lowest measured temperature.

The next piece of code calculates the mean square displacement (MSD) values:

In [ ]:
#############
## MSD fit
###########
msdfit_options = dict(InputWorkspace='intensity_vs_Q',
                                   Model='Gauss',
                                   XStart=0.3,
                                   XEnd=1.9,
                                   SpecMin=0,
                                   SpecMax=83536 - 83505,
                                   OutputWorkspace='msdfit_summary',
                                   ParameterWorkspace='msdfit_params',  # contains MSD values, among other data
                                   FitWorkspaces='msdfit_curves'
                                   )
MSDFit(**msdfit_options)  # http://docs.mantidproject.org/nightly/algorithms/MSDFit-v1.html

We make use of workspace `intensity_vs_Q` that was generated by `ElasticWindowMultiple`. We assume the intensity decays with the Debye-Waller model, here implemented through the fit function
[MsdGauss](http://docs.mantidproject.org/nightly/fitting/fitfunctions/MsdGauss.html). This fit function is selected by the `Model='Gauss'` line. Three models are available (
[Gauss](http://docs.mantidproject.org/nightly/fitting/fitfunctions/MsdGauss.html), 
[Peters](http://docs.mantidproject.org/nightly/fitting/fitfunctions/MsdPeters.html), and
[Yi](http://docs.mantidproject.org/nightly/fitting/fitfunctions/MsdYi.html)).

Here we selected to fit the decay of the intensity versus $Q$ over the whole available range of $Q$ values (from 0.3 to 1.9) and for all available temperatures. Workspace `intensity_vs_Q` has one spectrum for each temperature, and we have as many temperatures has runs. Thus, the range of spectra to use is from zero (first spectrum) to $83536 - 83505$ (last spectrum)

Algorithm [MSDFit](http://docs.mantidproject.org/nightly/algorithms/MSDFit-v1.html) produces three workspaces:

- `msdfit_summary`: a [MatrixWorkspace](http://www.mantidproject.org/Python_MatrixWorkspace_v2) containing selected fit parameters versus temperature. In our case, it contains two spectra corresponding to parameters `Heigth` and `MSD`
- `msdfit_params`: a [TableWorkspace](http://www.mantidproject.org/Python_ITableWorkspace_v2) containing detailed information for all fit parameters, as well as the goodness of fit ($\Chi^2$)
- `msdfit_curves`: a [WorkspaceGroup](http://www.mantidproject.org/Python_WorkspaceGroup_v2) containing one workspace for each temperature. Each workspace contains three spectra corresponding to (1)elastic intensty, (2) model fit, and (3) residuals.

If you are working with the `Script Window` of MantidPlot, these workspaces will be readily available in the workspace explorer. Otherwise, we have to save them to disk:

In [ ]:
##################################
## Save Workspaces to Nexus Files
##################################
SaveNexus(InputWorkspace='msdfit_summary', Filename='/tmp/msdfit_summary.nxs')
SaveNexus(InputWorkspace='msdfit_params', Filename='/tmp/msdfit_params.nxs')
SaveNexus(InputWorkspace='msdfit_curves', Filename='/tmp/msdfit_curves.nxs')

Three files will be saved in directory `/tmp`, since this is the directory we chose as our default save directory.

After saving, we can open MantidPlot and use algorithm [LoadNexus](http://docs.mantidproject.org/nightly/algorithms/LoadNexus-v1.html) to load these Nexus files onto workspaces.